<a href="https://colab.research.google.com/github/axel-sirota/nlp-and-transformers/blob/main/module3/NLPTransformers_Mod3Demo1_Attention_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducing Attention

© Data Trainers LLC. GPL v 3.0.

**Author:** Axel Sirota

Attention is one of the most groundbreaking ideas that revolutionized NLP and AI on the latest years. However, it is difficult to encounter a demo that is solely focused on attention... until now.

## Prep

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import gensim
from nltk.data import find
import nltk

nltk.download("word2vec_sample")

Let's define some helper functions we need:

* The softmax funciton definition for Numpy arrays
* An Embedder that transforms a list of words into its embedding representation according to `word2vec_sample` from the package `nltk`.

If you are unfamiliar with these concepts you are welcome to come to my other course **Implement Natural Language Processing for Word Embedding**

In [ ]:
def softmax(x, axis=0):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x))

In [ ]:
def get_word2vec_embedding(words):
    """
    Function that takes in a list of words and returns a list of their embeddings,
    based on a pretrained word2vec encoder.
    """
    word2vec_sample = str(find("models/word2vec_sample/pruned.word2vec.txt"))
    model = gensim.models.KeyedVectors.load_word2vec_format(
        word2vec_sample, binary=False
    )

    output = []
    words_pass = []
    for word in words:
        try:
            output.append(np.array(model.word_vec(word)))
            words_pass.append(word)
        except:
            pass

    embeddings = np.array(output)
    del model  # free up space again
    return embeddings, words_pass


## Attention 101: Dot product Attention

The idea behind attention is simple, if you take any word, like `Apple`, its meaning will change with respect with the other words in the sentence. For example below, In the first sentence Apple refers to the company and has strong relationship with coding and computer; on the second one refers to the fruit and therefore at most it would have relationship with eating, but not coding.

<figure>
<center>
<img src='https://www.dropbox.com/s/91xzqre8dpvxrux/sentence.png?raw=1' alt="drawing" width="350" />
<figcaption>Words relevance change with context</figcaption></center>
</figure>

  What I just spoke, is known as **Cross Attention**, because you will calculate the relationship of one word with respect to **all** the others in the sentence. In an image it would be:


<figure>
<center>
<img src='https://www.dropbox.com/s/ahn8ogriuzasa9a/attention_in_detail.png?raw=1'  />
<figcaption>Attention</figcaption></center>
</figure>

In code it is even easier, don't worry about the image above it will make sense as we evolve through the course. The really important part is the following:

$$
a_{ij} = f(h_i, s_{j})
$$

Where $a_{i,j}$ stands for the alignment of the word `h_i` with the output word `s_j`. The alignment may sound fancy, but it simply means how strongly connected those 2 words are in that sentence, like the Apple example!


The key is that the function $f$ can be anything. In the original paper, and the one we are implementing now it is the dot product, which you have probably seen before, and if not check the course I referenced before,  **Implement Natural Language Processing for Word Embedding**:

$$
a_{i,j} = dot product(h_i, s_j) = h_i^T*s_j
$$

So this means that for a given initial word, which is a row in the matrix we created, we have a Tensor of how aligned it is with that output word; we call that Tensor `c_k` or context vector.

And here comes the important stuff number 2, which is we take softmax to obtain weights, those wieghts will tell me for that input word how much weight (and importance) I should put into any output word. That is the attention matrix.

$$
z_j = softmax_k(c_{j,k})
$$

If we multiply this with the context vector of an encoder we have an empowered context memory tensor that can be fed into the decoder, as it is done in Transformers. We will implement all of this alongside this module

In [ ]:
def dot_product_attention(hidden_states, previous_state):

    # [T,d]*[d,N] -> [T,N]
    scores = np.matmul(previous_state, hidden_states.T)
    w_n = softmax(scores)

    # [T,N]*[N,d] -> [T,d]
    c_t = np.matmul(w_n, hidden_states)

    return w_n, c_t

Now we will use a helper function that will plot those attention weights I told you about

In [ ]:
def plot_attention_weight_matrix(weight_matrix, x_ticks, y_ticks):
    """Function that takes in a weight matrix and plots it with custom axis ticks"""
    plt.figure(figsize=(15, 7))
    ax = sns.heatmap(weight_matrix, cmap="Blues")
    plt.xticks(np.arange(weight_matrix.shape[1]) + 0.5, x_ticks)
    plt.yticks(np.arange(weight_matrix.shape[0]) + 0.5, y_ticks)
    plt.title("Attention matrix")
    plt.xlabel("Attention score")
    plt.show()

## Testing it out

Let's try with some words related to royalty and some related to food:

In [ ]:
words = ["king", "queen", "royalty", "food", "apple", "pear", "computers"]
word_embeddings, words = get_word2vec_embedding(words)
weights, _ = dot_product_attention(word_embeddings, word_embeddings)
plot_attention_weight_matrix(weights, words, words)

As you can see, this was successful! We could detect the relationships between apple, pear and a little less food; aas one cluster. Then another cluster of the royalty, and finally commputers alone, so it detected what it is supposed to! In the next demo we will implement other forms of attention, ie: changing that function `f`